In [22]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import tensorflow as tf
# from keras import layers


from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image

import tensorflow.keras.backend as K


from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random



In [23]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("Not enough GPU hardware devices available")
else:
    config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  1


In [24]:
# tf.debugging.set_log_device_placement(True)       # Will print the log information of every execution of tensorflow, mainly about 

# Setting the training and testing datasets 

ImageDataGenerator.flow_from_directory: Takes the path to a directory and generates batches of augmented data.

In [25]:
def my_crop(img, start_y, start_x, desired_height, desired_width):
  if K.image_data_format() == 'channels_last':
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy = desired_height #input desired output size
    dx = desired_width #input desired output size
    # start_y = (height-dy)//2
    # start_x = (width-dx)//2
    return img[start_y:start_y+dy, start_x:(dx+start_x), :]
  else:
      assert img.shape[0] == 3
      height, width = img.shape[1], img.shape[2]
      dy = desired_height #input desired output size
      dx = desired_width #input desired output size
      # start_y = (height-dy)//2
      # start_x = (width-dx)//2
      return img[:,start_y:start_y + dy, start_x:(dx + start_x)]

def crop_generator(batches, start_y, start_x, crop_height, crop_width):
    '''
    Take as input a Keras ImageGen (Iterator) and generate
    crops from the image batches generated by the original iterator
    '''
    # print("I'm in the crop_generataor function")
    while True:
      batch_x, batch_y = next(batches)  # In batch_x there are the images, in batch_y there are the labels
    #   print("hello")
    #   print('the shape of tensor batch_x is:', batch_x.shape)
    #   print('batch_y is:', batch_y)
      
      if K.image_data_format() == 'channels_last':
        batch_crops = np.zeros((batch_x.shape[0], crop_height, crop_width, 3))
      else:
        batch_crops = np.zeros((batch_x.shape[0], 3, crop_height, crop_width))
      for i in range(batch_x.shape[0]):
        #   print("THe shape of a single image before the cropping is: ", batch_x[i].shape)
          batch_crops[i] = my_crop(batch_x[i], start_y, start_x, crop_height, crop_width)
      yield (batch_crops, batch_y)

In [26]:
# batch_size = 64
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255) # Typically used for data augmentation. Here it is only rescaling the image pixels value to range 0 to 1 in decimals

train_generator = datagen.flow_from_directory(
    "../../Images/june5Night/Positive/",         # For home laptop
    # "../../../BottleStoodUp_atNight/Positive/",          # For work laptop
    batch_size=batch_size,
    shuffle = False,
    class_mode='input'
    
    )

Found 295 images belonging to 1 classes.


In [27]:
validation_generator = datagen.flow_from_directory(
    "../../Images/june5Night/Positive_val/",     # For home laptop
    # "../../../BottleStoodUp_atNight/Positive_val/",           # For work lpatop   
    batch_size=batch_size,
    shuffle = False,
    class_mode='input'
    )


Found 20 images belonging to 1 classes.


In [28]:
anomaly_generator = datagen.flow_from_directory(
    "../../Images/june5Night/Anomalies2.0/",         # For home laptop
    # "../../../BottleStoodUp_atNight/Anomalies2.0/",           # For work lpatpo
    batch_size=batch_size,
    shuffle = False,
    class_mode='binary'
    )


Found 10 images belonging to 1 classes.


In [29]:
test_set_generator = datagen.flow_from_directory(
    "../../Images/june5Night/Evaluation/",       # For home laptop
    # "../../../BottleStoodUp_atNight/Evaluation/",             # For work laptop  
    batch_size=batch_size,
    shuffle = False,
    class_mode='binary'
    )


Found 30 images belonging to 2 classes.


Creating the cropped images

In [30]:
HEIGHT = 160
WIDTH = 128

In [31]:
train_crops = crop_generator(train_generator, 65, 69, HEIGHT,WIDTH)      # top, left, height, width
val_crops = crop_generator(validation_generator, 65, 69, HEIGHT,WIDTH)      # top, left, height, width
anomaly_crops = crop_generator(anomaly_generator, 65, 69, HEIGHT,WIDTH)      # top, left, height, width
test_crops = crop_generator(test_set_generator, 65, 69, HEIGHT,WIDTH)      # top, left, height, width

In [32]:
print(type(train_crops))

<class 'generator'>


# FIlling the batches into lists

Grabbing only one image of one batch of the training set.

In [11]:
list_batches_training = []
for i in range(len(train_generator)):
    images_in_batch = next(train_crops)[0]
    print(images_in_batch.shape)
    list_batches_training.append(images_in_batch)

(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(32, 160, 128, 3)
(7, 160, 128, 3)


In [12]:
print(len(list_batches_training))

10


In [13]:
list_batches_val_pos = []
for i in range(len(validation_generator)):
    images_in_batch = next(val_crops)[0]
    print(images_in_batch.shape)
    list_batches_val_pos.append(images_in_batch)
print(len(list_batches_val_pos))

(20, 160, 128, 3)
1


In [14]:
list_batches_anomaly = []
for i in range(len(anomaly_generator)):
    images_in_batch = next(anomaly_crops)[0]
    print(images_in_batch.shape)
    list_batches_anomaly.append(images_in_batch)
print(len(list_batches_anomaly))

(10, 160, 128, 3)
1


In [15]:
list_batches_test = []
for i in range(len(test_set_generator)):
    images_in_batch = next(test_crops)[0]
    print(images_in_batch.shape)
    list_batches_test.append(images_in_batch)
print(len(list_batches_test))

(30, 160, 128, 3)
1


# Configuring the Autoencoder netwrok

In [16]:
latent_dim = 40  # Number of latent dimension parameters

input_img = Input(shape=(HEIGHT, WIDTH, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPool2D( (2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPool2D( (2, 2), padding='same')(x)

x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
x = MaxPool2D( (2, 2), padding='same')(x)
x = Conv2D(1, (3, 3), activation='relu', padding='same')(x)

shape_before_flattening = K.int_shape(x)
print(shape_before_flattening)
x = Flatten()(x)
x = Dense(80, activation='relu')(x)        # Activation function is supposed to be none, i.e the output is the same as the input

Z = Dense(latent_dim)(x)
print(K.int_shape(Z))

encoder = Model(input_img,Z)
encoder.summary()

2023-06-10 22:24:45.487360: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-06-10 22:24:45.511764: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2599990000 Hz
2023-06-10 22:24:45.512196: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f37f8000db0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-06-10 22:24:45.512210: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-06-10 22:24:45.623666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-10 22:24:45.624070: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3e72520 initialized for platform CUDA (this does not guarantee that XLA will

(None, 10, 8, 1)
(None, 40)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 64, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 32, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (Non

In [17]:
# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(Z)[1:])
x = Dense(10*8*4, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
# Expand to 784 total pixels
x = Dense(320, activation='sigmoid', name="original_decoder")(x)
x = Reshape((10,8,4),input_shape=(320,))(x)

x = Conv2DTranspose(3, (3, 3), padding='same')(x)
x = UpSampling2D( (2, 2))(x)
x = Conv2DTranspose(3, (3, 3), padding='same')(x)
x = UpSampling2D( (2, 2))(x)
x = Conv2DTranspose(3, (3, 3), padding='same')(x)
x = UpSampling2D( (2, 2))(x)
x = Conv2DTranspose(3, (3, 3), padding='same')(x)
x = UpSampling2D( (2, 2))(x)

# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
z_decoded = decoder(Z)

decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 40)]              0         
_________________________________________________________________
intermediate_decoder (Dense) (None, 320)               13120     
_________________________________________________________________
original_decoder (Dense)     (None, 320)               102720    
_________________________________________________________________
reshape (Reshape)            (None, 10, 8, 4)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 10, 8, 3)          111       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 20, 16, 3)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 20, 16, 3)         84  

In [18]:
# VAE model statement
ae = Model(input_img,z_decoded)
ae.compile(optimizer='adam', loss='mean_squared_error')

ae.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 64, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 32, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 16, 16)        0   

These are the weights of a specific layer of the encoder model, before training are:

In [19]:
layer_number = 1
print("quantity of layers in the model: ", len(encoder.layers))
print("The weigths in the", encoder.layers[layer_number].name, "is: ", encoder.layers[layer_number].weights)

quantity of layers in the model:  13
The weigths in the conv2d is:  [<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[ 0.04087575,  0.05895998, -0.00430713, ..., -0.08326497,
          -0.04953241, -0.03582999],
         [ 0.07479501, -0.00348064,  0.07686475, ...,  0.07988627,
           0.00465287,  0.06445852],
         [-0.04686734,  0.03561288, -0.02388179, ...,  0.01475448,
          -0.05125507, -0.00532942]],

        [[ 0.01826275, -0.00926737, -0.04323723, ..., -0.05338909,
          -0.05304727, -0.02992889],
         [ 0.01406843, -0.0106613 ,  0.08592266, ...,  0.08037448,
          -0.02010875,  0.07103062],
         [ 0.01101224,  0.0943874 ,  0.07306336, ..., -0.0041161 ,
          -0.08485003, -0.00857042]],

        [[-0.02545605,  0.07075962, -0.00906321, ...,  0.01940541,
          -0.03709448,  0.01455449],
         [ 0.07568547,  0.00201162, -0.05924144, ..., -0.03909023,
           0.02208322,  0.06256922],
         [-0.03270899,

These are the weights of a specific layer of the decoder model,before training are:

In [20]:
# layer_number = 1
# print("quantity of layers in the model: ", len(decoder.layers))
# print("The weigths in the", decoder.layers[layer_number].name, "is: ", decoder.layers[layer_number].weights)

These are the weights of a specific layer of the autoencoder model,before training are:

In [21]:
# layer_number = 1
# print("quantity of layers in the model: ", len(ae.layers))
# print("The weigths in the", ae.layers[layer_number].name, "is: ", ae.layers[layer_number].weights)

In [21]:
# from tensorflow.keras.callbacks import EarlyStopping
# run the model
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=5, mode='auto')
# history = ae.fit(train_generator, epochs=400, validation_data=validation_generator, verbose=1, shuffle = True)

history = ae.fit(x=list_batches_training,y=list_batches_training, epochs=300, validation_data=list_batches_val_pos, verbose=1, shuffle = True)

ValueError: Data cardinality is ambiguous:
  x sizes: 32, 32, 32, 32, 32, 32, 32, 32, 32, 7
  y sizes: 32, 32, 32, 32, 32, 32, 32, 32, 32, 7
Please provide data which shares the same first dimension.

Error matching the samples of the training and the validation sets, or something related.